In [1]:
from pathlib import Path
import polars as pl
import xlsxwriter

chrom_stat_dir: Path = Path('/mnt/eqa/zhangyuanfeng/methylation/evaluated/chrom_stats')
chrom_depth_stat_dir: Path = Path('/mnt/eqa/zhangyuanfeng/methylation/evaluated/chrom_depth_stats')

workbook  = xlsxwriter.Workbook('/mnt/eqa/zhangyuanfeng/methylation/evaluated/chrom_stats.xlsx')

store all info

In [2]:
female_chroms: list[str] =  ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10',
                              'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19',
                             'chr20', 'chr21', 'chr22', 'chrX', 'chrM']
male_chroms: list[str] = female_chroms + ['chrY']
female_with_controls: list[str] = female_chroms + ['lambda', 'pUC19']
male_with_controls: list[str] = male_chroms + ['lambda', 'pUC19']

In [3]:
df: pl.DataFrame
df = (pl.concat([pl.read_parquet(chrom_stat_dir / fname).with_columns(pl.lit(tag).alias('pipeline'))
                 for fname, tag in zip(['unified_informed.parquet.lz4',
                                        'unified_agnostic.parquet.lz4',
                                        'best_pipeline.parquet.lz4'],
                                        ['informed', 'agnostic', 'best_pipeline'])])
        .sort(by=['lab', 'label', 'rep', 'pipeline'])
        .drop('total')
        .with_columns(pl.sum_horizontal(male_with_controls).alias('total_with_wrong'))
        .with_columns(pl.when(pl.col('lab').str.contains_any(['RR1', 'RM1', 'NP1', 'MA']),
                              pl.col('label').str.contains_any(['F7', 'HF']))
                        .then(pl.sum_horizontal(male_chroms))
                        .when(pl.col('lab').str.contains_any(['RR1', 'RM1', 'NP1', 'MA']),
                              ~pl.col('label').str.contains_any(['F7', 'HF']))
                        .then(pl.sum_horizontal(female_chroms))
                        .when(~pl.col('lab').str.contains_any(['RR1', 'RM1', 'NP1', 'MA']),
                              pl.col('label').str.contains_any(['F7', 'HF']))
                        .then(pl.sum_horizontal(male_with_controls))
                        .otherwise(pl.sum_horizontal(female_with_controls))
                        .alias('total'))
        .with_columns((pl.col('total_with_wrong') - pl.col('total')).alias('wrong')))
df

lab,label,rep,count_type,fname,chr1,chr2,chr3,chr4,chr5,chr6,chr7,chr8,chr9,chr10,chr11,chr12,chr13,chr14,chr15,chr16,chr17,chr18,chr19,chr20,chr21,chr22,chrX,chrY,chrM,lambda,pUC19,pipeline,total_with_wrong,total,wrong
str,str,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,i64,i64
"""BS1""","""BC""","""1""","""reads""","""BS1_BC_1""",51512144,39049254,36995393,29966526,34423518,28042105,46548834,29828597,24212849,26804857,23001958,23195948,14902575,23083051,20217489,28739663,22146727,17240248,18595687,18508298,9536262,17580409,15550545,1201486,106736,761185,36419,"""best_pipeline""",601788763,600587277,1201486
"""BS1""","""BC""","""1""","""cytosine""","""BS1_BC_1""",4685840,4296304,3313069,2957693,3083191,2918008,3219411,2620271,2470205,2740084,2616322,2591828,1550119,1757950,1786682,2284444,2378872,1471416,2151504,1534475,876511,1234134,2528195,139118,870,6225,346,"""best_pipeline""",57213087,57073969,139118
"""BS1""","""BC""","""1""","""reads""","""BS1_BC_1""",51921470,40418670,37537951,31310140,34464524,28501234,45344300,30029001,22980075,27076830,22839239,23221579,15031290,23299049,19988571,32684327,23280571,14951076,17379747,17963147,7362397,17315422,13607231,877898,2081927,10230463,113653,"""informed""",611811782,610933884,877898
"""BS1""","""BC""","""1""","""cytosine""","""BS1_BC_1""",4493546,4228097,3233320,2888398,2946875,2858690,3080432,2523258,2337288,2670142,2549351,2538836,1495732,1697224,1670845,2141706,2297395,1347863,2061499,1472476,744980,1155348,2200541,19658,864,6226,346,"""informed""",54660936,54641278,19658
"""BS1""","""BC""","""2""","""reads""","""BS1_BC_2""",47824913,37685576,36061924,30346142,33564630,27339289,44376130,28621175,23152410,25254215,21468927,22158844,14837469,21856600,18849290,25734254,19283751,17487709,15797925,17175204,9293931,15296186,15428264,1058893,105113,745230,35637,"""best_pipeline""",570839631,569780738,1058893
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""RR1""","""T4""","""2""","""reads""","""RR1_T4_2""",8493337,6580537,4805638,4215368,4380065,4490168,5506619,3888165,4403792,4545850,4773927,4553927,2228459,3035449,2959344,4954498,5958569,1953664,6936141,3109093,2236321,2988866,2906052,481446,168505,17094863,374747,"""agnostic""",118023410,100072354,17951056
"""RR1""","""T4""","""2""","""cytosine""","""RR1_T4_2""",766808,575248,402858,366936,385637,385864,491863,341544,400513,397991,414209,389313,199515,262503,267379,435570,507103,171718,575918,251885,156817,259270,309582,36418,129,3494,96,"""best_pipeline""",8756181,8716173,40008
"""RR1""","""T4""","""2""","""reads""","""RR1_T4_2""",10788968,7602048,5394478,5138067,5120810,5159250,6456549,4591466,5184513,5705530,5589655,5359833,2636428,3452071,3534669,5650244,6769132,2289973,7822013,3366709,3440639,3333218,3936717,395786,8667,402309,8167,"""best_pipeline""",119137909,118331647,806262


In [4]:
df.write_excel(workbook=workbook, worksheet=workbook.add_worksheet('chrom_counts'),
               include_header=True, autofit=True)

In [5]:
dfs: list[pl.DataFrame] = []
for fname, tag in zip(['unified_informed.parquet.lz4',
                       'unified_agnostic.parquet.lz4',
                       'best_pipeline.parquet.lz4'],
                      ['informed', 'agnostic', 'best_pipeline']):
    tag_df: pl.DataFrame
    depth_df: pl.DataFrame
    chrom_df: pl.DataFrame

    tag_df = (pl.read_parquet(chrom_depth_stat_dir / fname)
                .with_columns(pl.lit(tag).alias('pipeline'),
                              pl.col('depth_bin').cast(pl.String)))
    dfs.append(tag_df)


(pl.concat(dfs)
   .sort('lab', 'label', 'rep', 'fname', 'chrom', 'depth_bin', 'count_type', 'pipeline')
   .write_excel(workbook=workbook, worksheet=workbook.add_worksheet('chrom_depth_counts'),
                include_header=True, autofit=True))

workbook.add_format({'font_size': 14, 'bold': True})

workbook.close()

In [13]:
stat_list = []
stat_df: pl.DataFrame = (pl.concat(dfs)
                           .sort('lab', 'label', 'rep', 'fname',
                                 'chrom', 'depth_bin',
                                 'count_type', 'pipeline'))

for fname in stat_df['fname'].unique():
    chroms: list[str] = [f'chr{i}' for i in range(1, 23)] + ['chrX', 'chrM']

    if not any(l in fname for l in {'RR1', 'RM1', 'NP1', 'HF'}):
        chroms += ['lambda', 'pUC19']

    if any(l in fname for l in {'F7', 'HF'}):
        chroms += ['chrY']

    for tag in ['informed', 'agnostic', 'best_pipeline']:
        tag_df: pl.DataFrame = stat_df.filter(pl.col('fname') == fname,
                                             pl.col('pipeline') == tag,
                                             pl.col('chrom').is_in(chroms))
        total_cytosine: int = tag_df.filter(pl.col('count_type') == 'cytosine')['count'].sum()
        total_depth: int = tag_df.filter(pl.col('count_type') == 'reads')['count'].sum()

        tag_tmp: dict[str, int] = {
            depth_bin: tag_df.filter(pl.col('count_type') == 'cytosine',
                                     pl.col('depth_bin') == depth_bin)['count'].sum()
            for depth_bin in tag_df['depth_bin'].unique()
        }
        
        stat_list.append({
            'fname': fname, 'pipeline': tag, 'total_cytosine': total_cytosine, 'total_depth': total_depth,
            **tag_tmp
        })

In [9]:
pl.DataFrame(stat_list).filter(pl.col('total_cytosine') > 0)

fname,pipeline,total_cytosine,total_depth,40–50x,.
str,str,i64,i64,i64,i64
"""BS1_D6_2""","""best_pipeline""",55592464,0,59161,55533303
"""PS1_T2_2""","""best_pipeline""",56197572,0,28053,56169519
"""RR1_T4_1""","""agnostic""",6400316,0,568417,5831899
"""RR1_T4_1""","""best_pipeline""",9633847,0,600747,9033100
"""BS1_F7_1""","""agnostic""",47064462,0,22040,47042422
…,…,…,…,…,…
"""EM4_T2_1""","""best_pipeline""",57642571,0,268056,57374515
"""BS2_T1_2""","""best_pipeline""",57754727,0,330083,57424644
"""RM1_T3_1""","""informed""",30043092,0,null,30043092


In [16]:
merged_stat: pl.DataFrame = (pl.DataFrame(stat_list)
                               .with_columns(pl.col('fname').str.head(3).alias('lab'),
                                             pl.col('fname').str.slice(offset=4, length=2).alias('label'),
                                             pl.col('fname').str.tail(n=1).alias('rep'))
                               .select('lab', 'label', 'rep', 'fname', 'pipeline',
                                       'total_cytosine', 'total_depth',
                                      '1x', '2x', '3x', '4x', '5–10x', '10–20x',
                                      '20–30x', '30–40x', '40–50x', '50–60x',
                                       '≥60x')
                               .fill_null(0)
                               .filter(pl.col('total_cytosine') > 0,
                                       pl.col('total_depth') > 0)
                               .sort(by=['lab', 'label', 'rep']))
merged_stat

lab,label,rep,fname,pipeline,total_cytosine,total_depth,1x,2x,3x,4x,5–10x,10–20x,20–30x,30–40x,40–50x,50–60x,≥60x
str,str,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""BS1""","""BC""","""1""","""BS1_BC_1""","""informed""",54641278,610933884,2737109,2349745,2981919,3491671,19420248,19411915,3614254,498729,70012,17430,48246
"""BS1""","""BC""","""1""","""BS1_BC_1""","""best_pipeline""",57073969,600587277,3056755,2714709,3424473,3966413,20893924,18932775,3261045,507744,119673,52819,143639
"""BS1""","""BC""","""2""","""BS1_BC_2""","""best_pipeline""",56755018,569780738,4395747,3573569,4159229,4448397,19820304,16103647,3173458,657554,179370,74519,169224
"""BS1""","""BC""","""2""","""BS1_BC_2""","""informed""",54428472,580696075,3943548,3114024,3656555,3969226,18631814,16720115,3503227,673251,130544,31654,54514
"""BS1""","""BL""","""1""","""BS1_BL_1""","""best_pipeline""",57471398,592996136,2061866,2002562,2756597,3477138,22825845,22461201,1498429,154694,68614,39078,125374
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""RR1""","""T4""","""1""","""RR1_T4_1""","""informed""",8481039,166986191,1474214,336722,290477,258953,1094749,1744539,1260262,903850,568961,294869,253443
"""RR1""","""T4""","""1""","""RR1_T4_1""","""agnostic""",6379626,162824914,0,0,0,258953,1094749,1744539,1260262,903850,568961,294869,253443
"""RR1""","""T4""","""2""","""RR1_T4_2""","""informed""",7776876,105141716,1596120,443791,383608,343631,1393504,1804860,985589,496384,202185,76180,51024


In [17]:
merged_stat.write_parquet('/mnt/eqa/zhangyuanfeng/methylation/evaluated/depth_stat.parquet.lz4',
                          compression='lz4')